## Analyse de sentiments

L'analyse de sentiments est une application classique de NLP. Le principe est de prédire les sentiments ou l'émotion contenue dans une phrase ou un texte : est-ce positif ? est-ce négatif ?

## Polarité et subjectivité

L'approche la plus directe est d'essayer de prendre en compte deux aspects :
- la polarité des mots, entre -1 et 1
- la subjectivité des mots, entre 0 et 1

Ces informations, bien que relatives, peuvent être automatiquement calculées à l'aide de TextBlob.

TextBlob s'installe de la façon suivante :
```
 conda install -c conda-forge textblob 
```

Il peut ensuite être facilement importé :

In [2]:
from textblob import TextBlob

Il est ensuite possible de récupérer polarité et subjectivité (sur texte anglais) avec les attributs `.polarity` et `.subjectivity`. Ou même les deux avec `.sentiment` :

In [3]:
print('Great: ', TextBlob('Great').sentiment)
print('Greater: ', TextBlob('Greater').sentiment)
print('Good: ', TextBlob('Good').sentiment)
print('Bad: ', TextBlob('Bad').sentiment)
print('Terrible: ', TextBlob('Terrible').sentiment)
print('Solid: ', TextBlob('Solid').sentiment)
print('Maybe: ', TextBlob('Maybe').sentiment)
print('Chicken: ', TextBlob('Chicken').sentiment)

Great:  Sentiment(polarity=0.8, subjectivity=0.75)
Greater:  Sentiment(polarity=0.5, subjectivity=0.5)
Good:  Sentiment(polarity=0.7, subjectivity=0.6000000000000001)
Bad:  Sentiment(polarity=-0.6999999999999998, subjectivity=0.6666666666666666)
Terrible:  Sentiment(polarity=-1.0, subjectivity=1.0)
Solid:  Sentiment(polarity=0.0, subjectivity=0.1)
Maybe:  Sentiment(polarity=0.0, subjectivity=0.0)
Chicken:  Sentiment(polarity=-0.6, subjectivity=0.95)


> Pour du français, voir le [textblob-fr](https://github.com/sloria/textblob-fr), probablement moins performant...

Ça fonctionne également sur des phrases complètes, ce qui donne une analyse de sentiment binaire, positif ou négatif par exemple :

In [4]:
print('Python:', TextBlob('Python is a cool and easy language').sentiment)
print('COBOL:', TextBlob('COBOL is a very old and complicated language').sentiment)

Python: Sentiment(polarity=0.39166666666666666, subjectivity=0.7416666666666667)
COBOL: Sentiment(polarity=-0.185, subjectivity=0.63)


À partir de ces résultats, on peut simplement appliquer un seuil à `0` :
- si la polarité de la phrase est supérieure à 0, le commentaire est positif
- sinon, le commentaire est négatif

On a donc bien effectué de l'analyse de sentiment, et prédit le caractère positif ou négatif d'un commentaire.

**Exercice :**

Sur le jeu de données `amazon_reviews.csv`, utiliser la polarité pour prédire si la revue est positive ou négative.

Calculer l'accuracy de la prédiction.

> Aide : le séparateur est `'|'`, l'accuracy peut se calculer à l'aide de `sklearn.metrics.accuracy_score`

In [5]:
import pandas as pd

df = pd.read_csv('data/amazon_reviews.csv', sep='|')


## Descripteurs et classification

Une autre méthode pour effectuer de l'analyse de sentiments, c'est d'utiliser les descripteurs, associés à des algorithmes de classification :
- une première étape de preprocessing et calcul des descripteurs (BOW ou TF-IDF par exemple)
- une classification binaire sur ces descripteurs
- éventuellement une optimisation des hyperparamètres

### Calcul du TF-IDF

La première étape est donc le calcul du TF-IDF :

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

stops = stopwords.words('english')

vectorizer = TfidfVectorizer(stop_words=stops)

# Calcul du TF-IDF des review amazon
tfidf = vectorizer.fit_transform(df['review']).toarray()

### Application d'un modèle de classification simple : Naive Bayes et Régression Logistique

Nous allons maintenant appliquer deux modèles de Machine Learning classique : le Naive Bayes et la Régression logistique.

Ces derniers sont implémentés dans scikit-learn et peuvent être importés comme tel :
```python
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
```

Avant cela, il faut séparer les données en deux groupes :
- le groupe `train` sur lequel sera entrainé le modèle : il représente en général 80 % des données
- le groupe `test` sur lequel sera évalué le modèle : il représente en général 20 % des données

Ce découpage peut se faire automatiquement à l'aide de la fonction `train_test_split` de scikit-learn :
```python 
from sklearn.model_selection import train_test_split
```

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tfidf, df['class'], test_size=0.2, random_state=0)

Il faut ensuite instancier puis entrainer le modèle sur le jeu `train`, à l'aide de la méthode `.fit()` :

In [8]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

# On instancie les modèles
nb = GaussianNB()
lr = LogisticRegression()

# On entraine les modèles
nb.fit(X_train, y_train)
lr.fit(X_train, y_train)

LogisticRegression()

Enfin, il est possible de prédire les résultats sur la partie `test` à l'aide de la méthode `.predict()`. Puis on peut finalement évaluer les performances de notre modèle, avec une métrique telle que l'`accuracy_score` :

In [9]:
from sklearn.metrics import accuracy_score

pred_nb = nb.predict(X_test)
pred_lr = lr.predict(X_test)

print('accuracy du modèle Naive Bayes :', accuracy_score(y_test, pred_nb))
print('accuracy du modèle Logistic Regression :', accuracy_score(y_test, pred_lr))

accuracy du modèle Naive Bayes : 0.5895
accuracy du modèle Logistic Regression : 0.859


Que pensez-vous de ces résultats, en comparaison avec l'utilisation de la polarité des mots ?

**Exercice :**

Effectuer une analyse de sentiments par classification binaire sur le jeu de données `IMDB_dataset.csv` :
- D'abord faire une extraction de descripteurs (BOW et/ou TF-IDF)
- Séparer les données
- Entrainer un modèle de classification binaire (e.g. LogisticRegression, SVM, RandomForestClassifier...)
- Evaluer les performances du modèle sélectionné

In [10]:
df = pd.read_csv('data/IMDB_dataset.csv')
df['class'] = (df['sentiment']=='positive').astype(int)
df.head()

,review,sentiment,class
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1
